# Crawl Brant Gardner Book of Mormon Minutes from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup, Tag
from models.crawl_utils import get_page, save_page

In [ ]:
# config
host = 'https://archive.bookofmormoncentral.org/content/book-mormon-minute'
base_dir = '../data/raw/bgbmm'
bs_parser = 'html.parser'
delay_seconds = 5

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    

In [ ]:



def extract_anchors(host, html):
    # Parse the HTML content
    soup = BeautifulSoup(html, 'html.parser')
    # Find all span tags with class "field-content" that contain anchor tags
    span_tags = soup.find_all('span', class_='field-content')
    # Extract anchor tags within each span tag
    hrefs = []
    for span_tag in span_tags:
        anchor_tag = span_tag.find('a', href = True)
        if not isinstance(anchor_tag, Tag):
            continue
        href = anchor_tag.get('href')
        href = urljoin(host, href)
        hrefs.append(href)
    return hrefs

In [ ]:
def get_path(url):
    if not url.endswith('/'):
        url += '/'
    path_components = urlparse(url).path.split('/')
    return os.path.join(base_dir, f"{path_components[-2]}.json")

In [ ]:
bgbmm_links = []

status_code, html = get_page(host, delay_seconds)
if status_code != 200:
    print("Error!", status_code)
hrefs = extract_anchors(host, html)
if len(hrefs) != 0:
    bgbmm_links.extend(hrefs)
    
print(bgbmm_links[:5])
len(bgbmm_links)

In [ ]:
for url in bgbmm_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")